In [2]:
import graspy as gp
import glob
import os, sys
import numpy as np
import pandas as pd
import multiprocessing as multiproc
import dask
from dask.distributed import Client, progress
import dask.dataframe as ddf
from scipy.stats import mannwhitneyu
from matplotlib.pyplot import savefig
import networkx as nx

In [90]:
gr = nx.read_graphml('./data/G.graphml')
A = nx.to_numpy_matrix(gr)
node_hemi = nx.get_node_attributes(gr, 'Hemisphere')
node_pair = nx.get_node_attributes(gr, 'Pair')
n=A.shape[0]

In [91]:
def make_edgecomm_from_dict(dict_nodes):
    C = np.zeros((len(dict_nodes), len(dict_nodes)))
    for i, (keyi, vali) in enumerate(dict_nodes.items()):
        for j, (keyj, valj) in enumerate(dict_nodes.items()):
            if i != j:
                if vali == valj:
                    C[i,j] = 1
                else:
                    C[i,j] = 2
    return C

In [112]:
Rowmtx = np.zeros((n,n))
Colmtx = np.zeros((n,n))
vtx_order = node_hemi
for i in range(1, n+1):
    Rowmtx[i-1,:] = i
    Colmtx[:,i-1] = i
    
C_hemi = make_edgecomm_from_dict(node_hemi)
C_pair = make_edgecomm_from_dict(node_pair)
    
A_df = pd.DataFrame({"Value": np.array(A).flatten(), 
                     "Row": Rowmtx.astype(int).flatten(),
                     "Column": Colmtx.astype(int).flatten(),
                     "Homophilic": C_hemi.astype(int).flatten(),
                     "Homotopic": C_pair.astype(int).flatten()})

A_df.to_csv('./data/dros_edges.csv')

In [105]:
def mww(G, C, alternative='two-sided'):
    A = G[C == 1]
    B = G[C == 2]
    return(mannwhitneyu(A, B, alternative=alternative)[1])

In [110]:
mww(np.array(A), C_hemi, alternative='greater')

0.0

In [111]:
mww(np.array(A), C_pair, alternative='greater')

0.0